# Import necessary libraries

In [143]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import os
from sklearn.cluster import KMeans
import folium 
print("folium imported")
from geopy.geocoders import Nominatim 
print("geopy imported")
import matplotlib.cm as cm
import matplotlib.colors as colors


folium imported
geopy imported


# Scan through the table and obtain the required data

In [144]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = requests.get(url).text
soup = BeautifulSoup(req, 'xml')
table=soup.find('table')
list_of_column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = list_of_column_names)
for tr in table.find_all('tr'):
    row_data=[]
    for td_cell in tr.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


# Clean the data as required

In [145]:
df=df[df['Borough']!='Not assigned']
tdf=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
tdf=tdf.reset_index(drop=False)
tdf.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)
dfm = pd.merge(df, tdf, on='Postalcode')
dfm.drop(['Neighborhood'],axis=1,inplace=True)
dfm.drop_duplicates(inplace=True)
dfm.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)
dfm.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [146]:
dfm.shape

(103, 3)

# 2nd submission starts here

In [149]:
#since I was not able to download the package needed
def geocode(zipcode):
    latandlong_coords = None
    while(latandlong_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(zipdode))
        latandlong_coords = g.latandlong
    lat = latandlong_coords[0]
    long = latandlong_coords[1]
    return lat,long

In [150]:
gdf=pd.read_csv('http://cocl.us/Geospatial_data')

In [151]:
gdf.head()
gdf.rename(columns={'Postal Code':'Postalcode'},inplace=True)
gmerged = pd.merge(gdf, dfm, on='Postalcode')
gdata=gmerged[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]

In [152]:
gdata.head(100)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
95,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201
96,M9L,North York,Humber Summit,43.756303,-79.565963
97,M9M,North York,"Humberlea, Emery",43.724766,-79.532242
98,M9N,York,Weston,43.706876,-79.518188


# 3rd submissino starts here

In [153]:
toronto_data=gdata[gdata['Borough'].str.contains("Toronto")]
toronto_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [154]:
CLIENT_ID = 'X1DYZ0HYWBJ03LI5ZNQSD4WWQRKQNSTSJR32HGMU2NSXZQXF'
CLIENT_SECRET = 'HA2EKBWDBFE3EG5YHPKFH2FB4KKSQABHX5GQK1ZAE35MQY2Y'
VERSION = '20200824'

In [155]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


In [161]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [162]:
def commonvenues(row, top_venues):
    rows = row.iloc[1:]
    sorted_rows = rows.sort_values(ascending=False)
    return sorted_rows.index.values[0:top_venues]
top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for item in np.arange(top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(item+1, indicators[item]))
    except:
        columns.append('{}th Most Common Venue'.format(item+1))
sorted_neighborhoods_venues = pd.DataFrame(columns=columns)
sorted_neighborhoods_venues['Neighborhood'] = toronto_grouped['Neighborhood']
for item in np.arange(toronto_grouped.shape[0]):
    sorted_neighborhoods_venues.iloc[item, 1:] = commonvenues(toronto_grouped.iloc[item, :], top_venues)
#sorted_neighborhoods_venues.head()

In [163]:
k_clusters = 7
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
#kmeans.labels_[0:10]
sorted_neighborhoods_venues.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = toronto_data
toronto_merged = toronto_merged.join(sorted_neighborhoods_venues.set_index('Neighborhood'), on='Neighborhood')
#toronto_merged.head()
address = 'Toronto, CA'
g_locator = Nominatim(user_agent="ny_explorer")
location = g_locator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('lattitue and longitude of Manhattan are {}, {}.'.format(latitude, longitude))

lattitue and longitude of Manhattan are 43.6534817, -79.3839347.


In [164]:
map_clusters = folium.Map(location=[lat, longitude], zoom_start=11)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)      
map_clusters